In [13]:
from PrismFileReader import PrismFileReader
from PrismModule import PrismModule
import random
import os

failstateVar = 'failstate'
succProbVar = 'p'
succProb = 0.8

def diffLists(l1,l2):
    l1s = set(l1)
    l2s = set(l2)
    res = l1s.symmetric_difference(l2s)
    return list(res)

def sample(l,n):
    return random.sample(l,n)

def createStateString(variable,s):
    ss = '('+variable+'='+str(s)+')'
    return ss

def pickStates(previousStatesList,numToPick,currentStatesList):
    statesPicked = []
    n = numToPick
    if previousStatesList is not None:
        statesPicked = previousStatesList[0:n]
        n = n - len(statesPicked)
    if n> 0:
        statesToPickFrom = diffLists(statesPicked,currentStatesList)
        newStatesPicked = sample(statesToPickFrom,n)
        statesPicked = statesPicked + newStatesPicked
    return statesPicked

def ensureDir(dirLoc):
    if not os.path.exists(dirLoc):
        os.makedirs(dirLoc)

def getTermWidth():
    rows, columns = os.popen('stty size', 'r').read().split()
    width = int(columns)
    return width

def printC(string,width):
    #width = os.get_terminal_size().columns
    
    
    print(string.center(width))

    
def linInd(x,y,xside,yside):
    return y*yside + x

def stateString(varname,varvalue):
    return '('+varname+'='+str(varvalue)+')'

def stateLabel(varname,varvalue):
    return varname+str(varvalue)

def addFailstateConstant(pfr):
    if failstateVar not in pfr.constants:
        pfr.addConstant(failstateVar,'int','-1')
    return pfr

def addSuccProbConstant(pfr):
    if succProbVar not in pfr.constants:
        pfr.addConstant(succProbVar,'double',str(succProb))
    return pfr

def checkModInPFR(pfr,modName):
    addName = True
    if pfr.moduleNames is not None:
        if len(pfr.moduleNames) > 0:
            if modName in pfr.moduleNames:
                addName = False
    else:
        pfr.moduleNames=[]
    if addName:
        pfr.moduleNames.append(modName)
    return pfr    
        

In [14]:
def createWarehouseActions(gpfr,gmod,xside,yside,varname):
    scount = 0 
    for x in range(xside):
        for y in range(yside):
            scount = linInd(x,y,xside,yside)
            svar = stateLabel(varname,scount)
            sstr = stateString(varname,scount)
            if gpfr.labels is None:
                gpfr.addLabel(svar,sstr,gmod.variables)
            if not svar in gpfr.labels:
                gpfr.addLabel(svar,sstr,gmod.variables)
            xx = min(x+1,xside-1)
            yy = min(y+1,yside-1)
            _x = max(x-1,0)
            _y = max(y-1,0)
            if y == 0 or y == (yside-1) or x==0 or x==(xside-1):
                combs = [(x,yy),(x,_y),(xx,y),(_x,y)]
            else:
                combs = [(xx,y),(_x,y)]
            print (x,y)
            print combs
            raw_input()
            for xy in combs:
                xp = xy[0]
                yp = xy[1]
                if(xp == x and yp == y):
                    continue
                pcount = linInd(xp,yp,xside,yside)
                pstr = stateString(varname,pcount)
                pvar = stateLabel(varname,pcount)
                if not pvar in gpfr.labels:
                    gpfr.addLabel(pvar,pstr,gmod.variables)
                #pa = mod.createAction('s1_s2',{'"time"':1.0},'(s=0)',['(s=1)','(s=2)'],['p','1-p'])
                #pa = gmod.createAction(svar+'_'+pvar,{'"time"':1.0},sstr,[pstr,'(s=failstate)'],['p','1-p'])
                pa = gmod.createAction(svar+'_'+pvar,{'"time"':1.0},sstr,pstr,'1.0')
                #print pa
                gmod.actions.append(pa)
    return [gmod,gpfr]

In [15]:
def addWarehouseLinesModule(gridPfr,modName,xside,yside,numFailStates):
    #check if we have p and failstate 
    gridPfr = addFailstateConstant(gridPfr)
    gridPfr = addSuccProbConstant(gridPfr)
    gridPfr = checkModInPFR(gridPfr,modName)
    gmod = PrismModule(modName,[],gridPfr.constants)
    varname = 's'
    rewName = '"time"'
    gmod.addVariable(varname,gridPfr.constants[failstateVar],xside*yside,0,0)
    if gridPfr.rewardNames is None:
        gridPfr.rewardNames = []
    if rewName not in gridPfr.rewardNames:
        gridPfr.rewardNames.append(rewName)
    if gridPfr.modVars is None:
        gridPfr.modVars = []
    [gmod,gridPfr] = createWarehouseActions(gridPfr,gmod,xside,yside,varname)
    gridPfr.modVars.append(gmod)
    return gridPfr

In [16]:
def createGridActions(gpfr,gmod,xside,yside,varname):
    scount = 0 
    for x in range(xside):
        for y in range(yside):
            scount = linInd(x,y,xside,yside)
            svar = stateLabel(varname,scount)
            sstr = stateString(varname,scount)
            if gpfr.labels is None:
                gpfr.addLabel(svar,sstr,gmod.variables)
            if not svar in gpfr.labels:
                gpfr.addLabel(svar,sstr,gmod.variables)
            xx = min(x+1,xside-1)
            yy = min(y+1,yside-1)
            _x = max(x-1,0)
            _y = max(y-1,0)
            combs = [(x,yy),(x,_y),(xx,y),(_x,y)]

            for xy in combs:
                xp = xy[0]
                yp = xy[1]
                if(xp == x and yp == y):
                    continue
                pcount = linInd(xp,yp,xside,yside)
                pstr = stateString(varname,pcount)
                pvar = stateLabel(varname,pcount)
                if not pvar in gpfr.labels:
                    gpfr.addLabel(pvar,pstr,gmod.variables)
                #pa = mod.createAction('s1_s2',{'"time"':1.0},'(s=0)',['(s=1)','(s=2)'],['p','1-p'])
                #pa = gmod.createAction(svar+'_'+pvar,{'"time"':1.0},sstr,[pstr,'(s=failstate)'],['p','1-p'])
                pa = gmod.createAction(svar+'_'+pvar,{'"time"':1.0},sstr,pstr,'1.0')
                #print pa
                gmod.actions.append(pa)
    return [gmod,gpfr]

In [17]:
def addGridLinesModule(gridPfr,modName,xside,yside,numFailStates):
    #check if we have p and failstate 
    gridPfr = addFailstateConstant(gridPfr)
    gridPfr = addSuccProbConstant(gridPfr)
    gridPfr = checkModInPFR(gridPfr,modName)
    gmod = PrismModule(modName,[],gridPfr.constants)
    varname = 's'
    rewName = '"time"'
    gmod.addVariable(varname,gridPfr.constants[failstateVar],xside*yside,0,0)
    if gridPfr.rewardNames is None:
        gridPfr.rewardNames = []
    if rewName not in gridPfr.rewardNames:
        gridPfr.rewardNames.append(rewName)
    if gridPfr.modVars is None:
        gridPfr.modVars = []
    [gmod,gridPfr] = createGridActions(gridPfr,gmod,xside,yside,varname)
    gridPfr.modVars.append(gmod)
    return gridPfr

In [19]:
gridPfr = PrismFileReader(None)
modName = 'grid'
xside = 4  
yside = 4 
numFailStates = 0
gridPfr = addWarehouseLinesModule(gridPfr,modName,xside,yside,numFailStates)

an empty file
Initializing Module grid
(0, 0)
[(0, 1), (0, 0), (1, 0), (0, 0)]

(0, 1)
[(0, 2), (0, 0), (1, 1), (0, 1)]

(0, 2)
[(0, 3), (0, 1), (1, 2), (0, 2)]

(0, 3)
[(0, 3), (0, 2), (1, 3), (0, 3)]

(1, 0)
[(1, 1), (1, 0), (2, 0), (0, 0)]

(1, 1)
[(2, 1), (0, 1)]

(1, 2)
[(2, 2), (0, 2)]

(1, 3)
[(1, 3), (1, 2), (2, 3), (0, 3)]

(2, 0)
[(2, 1), (2, 0), (3, 0), (1, 0)]

(2, 1)
[(3, 1), (1, 1)]

(2, 2)
[(3, 2), (1, 2)]

(2, 3)
[(2, 3), (2, 2), (3, 3), (1, 3)]

(3, 0)
[(3, 1), (3, 0), (3, 0), (2, 0)]

(3, 1)
[(3, 2), (3, 0), (3, 1), (2, 1)]

(3, 2)
[(3, 3), (3, 1), (3, 2), (2, 2)]

(3, 3)
[(3, 3), (3, 2), (3, 3), (2, 3)]



In [20]:
lines = gridPfr.createFileLinesToWrite()
gridPfr.writeLinesToFile(lines,'warehouse.prism')